In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np

# Define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 classes for CIFAR-10

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define data transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download CIFAR-10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

# Initialize the network and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training loop
for epoch in range(5):  # You can adjust the number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / (i + 1)}")

print('Finished Training')

# Save the model
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

# Testing the model on the test dataset (you need to define testloader similarly to trainloader)
# ... code for test dataset loading ...

# Load the model
# Define the test dataset and data loader
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

net = Net()
net.load_state_dict(torch.load(PATH))

# Evaluate the model and calculate accuracy
correct = 0
total = 0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print(f"Accuracy of the network on the test images: {100 * correct / total}%")

for i in range(10):
    print(f"Accuracy of class {i}: {100 * class_correct[i] / class_total[i]}%")


Files already downloaded and verified
Epoch 1, Loss: 1.712259569631815
Epoch 2, Loss: 1.328541271419525
Epoch 3, Loss: 1.1873809371569752
Epoch 4, Loss: 1.0925721754463762
Epoch 5, Loss: 1.0132014092504977
Finished Training
Files already downloaded and verified
Accuracy of the network on the test images: 62.24%
Accuracy of class 0: 65.5%
Accuracy of class 1: 65.4%
Accuracy of class 2: 49.4%
Accuracy of class 3: 42.5%
Accuracy of class 4: 63.4%
Accuracy of class 5: 36.0%
Accuracy of class 6: 79.3%
Accuracy of class 7: 64.8%
Accuracy of class 8: 78.7%
Accuracy of class 9: 77.4%
